# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
import pandas as pd

# Define the column names
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']

# Load the dataset
file_path = 'training.1600000.processed.noemoticon.csv' 
data = pd.read_csv(file_path, encoding='ISO-8859-1', names=column_names)
df = data.sample(20000)

# Display the first few rows of the dataframe
df.head()

,target,id,date,flag,user,text
595216,0,2218712325,Wed Jun 17 23:00:15 PDT 2009,NO_QUERY,Sabri32,"@Solarica96 oh ok, yeah there is absolutely no..."
1365402,4,2049951090,Fri Jun 05 17:47:40 PDT 2009,NO_QUERY,essencerenata,@BlackDreams glad to hear that the shoot went ...
88689,0,1754986861,Sun May 10 07:57:15 PDT 2009,NO_QUERY,joannasas,My bestfriend's getting addicted to Facebook's...
957234,4,1825414362,Sun May 17 05:36:50 PDT 2009,NO_QUERY,CharlieAndMe,@hotmamabrew we all need some support in our l...
756469,0,2288736919,Mon Jun 22 19:33:06 PDT 2009,NO_QUERY,consciousdear88,Still can't make him happy.


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [2]:
def process_text(text):
    # Clean up the text
    text = clean_up(text)
    
    # Tokenize the text
    tokens = tokenize(text)
    
    # Stem and Lemmatize the tokens
    tokens = stem_and_lemmatize(tokens)
    
    # Remove Stop Words from the tokens
    tokens = remove_stopwords(tokens)
    
    return tokens

In [3]:
import re

def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    # Removing URLs
    s = re.sub(r'http\S+|www.\S+', '', s)
    
    # Replacing numbers and special characters with a whitespace
    s = re.sub(r'[^a-zA-Z\s]', ' ', s)
    
    # Converting to lowercase
    s = s.lower()
    
    return s

In [4]:
import nltk

def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    
    # Using NLTK's word_tokenize function to tokenize the string
    return nltk.word_tokenize(s)

In [5]:
from nltk.stem import PorterStemmer, WordNetLemmatizer



def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    
    # Initializing the stemmer and lemmatizer
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    stemmed = [stemmer.stem(word) for word in l]
    lemmatized = [lemmatizer.lemmatize(word) for word in l]

    
    return lemmatized


In [6]:
from nltk.corpus import stopwords

def remove_stopwords(tokens):
    """
    Remove English stopwords from a list of strings.

    Args:
        tokens: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    # Getting the English stop words from nltk
    stop_words = set(stopwords.words('english'))
    
    # Returning a list of tokens which are not stop words
    return [token for token in tokens if token not in stop_words]


In [7]:
# Applying the processing function to the 'text' column and creating a new 'text_processed' column
df['text_processed'] = df['text'].apply(process_text)

In [8]:
df.head()

,target,id,date,flag,user,text,text_processed
595216,0,2218712325,Wed Jun 17 23:00:15 PDT 2009,NO_QUERY,Sabri32,"@Solarica96 oh ok, yeah there is absolutely no...","[solarica, oh, ok, yeah, absolutely, way, mine..."
1365402,4,2049951090,Fri Jun 05 17:47:40 PDT 2009,NO_QUERY,essencerenata,@BlackDreams glad to hear that the shoot went ...,"[blackdreams, glad, hear, shoot, went, great, ..."
88689,0,1754986861,Sun May 10 07:57:15 PDT 2009,NO_QUERY,joannasas,My bestfriend's getting addicted to Facebook's...,"[bestfriend, getting, addicted, facebook, pet,..."
957234,4,1825414362,Sun May 17 05:36:50 PDT 2009,NO_QUERY,CharlieAndMe,@hotmamabrew we all need some support in our l...,"[hotmamabrew, need, support, life, always, nic..."
756469,0,2288736919,Mon Jun 22 19:33:06 PDT 2009,NO_QUERY,consciousdear88,Still can't make him happy.,"[still, make, happy]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [9]:
import itertools

# Combine all the words in 'text_processed' column
all_words = list(itertools.chain(*df['text_processed']))


In [10]:
import nltk

# Calculate the frequency distribution of all words
freq_dist = nltk.FreqDist(all_words)

In [11]:
# Select the top 5,000 words
top_words = [word for word, count in freq_dist.most_common(5000)]

In [12]:
print(top_words[:10])

['day', 'wa', 'good', 'get', 'like', 'go', 'u', 'got', 'today', 'quot']


### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [13]:
def build_features(document, bag_of_words):
    document_words = set(document)
    features = {}
    for word in bag_of_words:
        features[word] = (word in document_words)
    return features


In [14]:
feature_set = [(build_features(doc, top_words), (target == 4)) for index, (doc, target) in df[['text_processed', 'target']].iterrows()]


In [15]:
# feature_set

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [16]:
# Define the proportion of data to be used for training
train_proportion = 0.8
train_size = int(len(feature_set) * train_proportion)

train_set, test_set = feature_set[:train_size], feature_set[train_size:]


In [17]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [18]:
classifier.show_most_informative_features(50)

Most Informative Features
                headache = True            False : True   =     24.4 : 1.0
                     sad = True            False : True   =     24.4 : 1.0
                   upset = True            False : True   =     15.5 : 1.0
                    died = True            False : True   =     15.3 : 1.0
           unfortunately = True            False : True   =     13.3 : 1.0
                     vip = True             True : False  =     13.1 : 1.0
                    poor = True            False : True   =     13.1 : 1.0
                   dying = True            False : True   =     12.9 : 1.0
                    nose = True            False : True   =     12.9 : 1.0
                     rip = True            False : True   =     12.9 : 1.0
                  throat = True            False : True   =     12.9 : 1.0
               hilarious = True             True : False  =     12.4 : 1.0
                   smile = True             True : False  =     12.3 : 1.0

In [25]:
print("Balance in the whole dataset:")
print(data['target'].value_counts())
print(df['target'].value_counts())

Balance in the whole dataset:
target
0    800000
4    800000
Name: count, dtype: int64
target
0    10042
4     9958
Name: count, dtype: int64


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [26]:
# Calculate the accuracy of the classifier on the test set
accuracy = nltk.classify.accuracy(classifier, test_set)
print(f"Model Accuracy: {accuracy}")

Model Accuracy: 0.72575


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [27]:
for num_features in [3000, 4000, 5000, 6000, 7000]:
    # Select the top 'num_features' features
    top_words = [word for word, count in freq_dist.most_common(num_features)]
    
    # Build the feature set
    feature_set = [(build_features(doc, top_words), (target == 4)) 
                   for index, (doc, target) in df[['text_processed', 'target']].iterrows()]
    
    # Split the dataset and train the model
    train_set, test_set = feature_set[:train_size], feature_set[train_size:]
    classifier = nltk.NaiveBayesClassifier.train(train_set)
    
    # Calculate and print the accuracy
    accuracy = nltk.classify.accuracy(classifier, test_set)
    print(f"Number of Features: {num_features}, Model Accuracy: {accuracy}")


Number of Features: 3000, Model Accuracy: 0.728
Number of Features: 4000, Model Accuracy: 0.7285
Number of Features: 5000, Model Accuracy: 0.72575
Number of Features: 6000, Model Accuracy: 0.727
Number of Features: 7000, Model Accuracy: 0.72825


## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [22]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [23]:
# your code here